In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics.pairwise import rbf_kernel 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import time
from sklearn.calibration import calibration_curve
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ndcg_score
from sklearn.metrics import ndcg_score
from sklearn.metrics import top_k_accuracy_score
from ncon import ncon
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import warnings

In [2]:
def effectivity_at_top_k(y_true, y_pred, k=1):
    return np.mean(y_true[np.argsort(y_pred)[::-1][:k]])

In [3]:
def outersum(a,b):
    return np.outer(a, np.ones(len(b))) + np.outer(np.ones(len(a)), b)

def outersum_deriv(a,b):
    return np.ones(len(a)), np.ones(len(b))

def maxoutersum(a,b,α):
    return np.log(outersum(np.exp(α*a), np.exp(α*b)))/α

def maxoutersum_deriv(a,b,α):
    D = 1./(outersum(np.exp(α*a), np.exp(α*b)))
    return (D.transpose()*np.exp(α*a)).transpose(), D*np.exp(α*b)

In [4]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=3)

In [5]:
def pr_auc_score(true, pred, r=None):
    lr_precision, lr_recall, _ = precision_recall_curve(true, pred)
    if r is not None:
        r_ = (1/np.nanmean(true))-1  # ratio negatives/positives
        #print(r,r_)
        lr_precision = lr_precision/(lr_precision+r/r_*(1-lr_precision))
    return auc(lr_recall, lr_precision)

def metric_(true, pred, metric): #metrics such as auc throw error in case of one class available.
    if metric==ndcg_score:
        true=[true]
        pred=[pred]
    #return metric(true, pred)
    try:
        warnings.filterwarnings("ignore")
        return metric(true, pred)
    except:
        warnings.filterwarnings("default")
        return np.nan
def evaluate_per_row(true, pred, metric):
    return np.nanmean([metric_(true[i,:][~np.isnan(true[i,:])], pred[i,:][~np.isnan(true[i,:])],metric) for i in range(true.shape[0])])
def evaluate_per_column(true, pred, metric):
    return evaluate_per_row(true.transpose(), pred.transpose(), metric)


def evaluate_direct(Y_test, P):
    resprflat = []
    resprperrow = []
    resndcgperrow =[]
    for stage in range(4):
        true, pred = copy.deepcopy(Y_test[:,:,stage]), copy.deepcopy(P[:,:,stage])
        
        prflat = pr_auc_score(true[~np.isnan(true)], pred[~np.isnan(true)], r=30)
        resprflat.append(prflat)
    
    print("& {:.2} & {:.2} & {:.2} & {:.2}  \\\\".format(resprflat[0],resprflat[1],resprflat[2],resprflat[3]))
    
    return resprflat
    
def evaluate_conditional(Y_test, P):
    resprflat = []
    resprperrow = []
    resndcgperrow =[]    
    for stage in range(1,4):
        true, pred = copy.deepcopy(Y_test[:,:,stage]), copy.deepcopy(P[:,:,stage]/P[:,:,stage-1])
        prev = copy.deepcopy(Y_test[:,:,stage-1])
        
        true[prev==0]=np.nan
        pred[prev==0]=np.nan

        
        prflat = pr_auc_score(true[~np.isnan(true)], pred[~np.isnan(true)],r=30)
        resprflat.append(prflat)
        
    print("& {:.2} & {:.2} & {:.2}  \\\\".format(resprflat[0],resprflat[1],resprflat[2]))
    
    return [np.nan] + resprflat
    
        

In [6]:
def visualizeprogression(m, Ltrain, Lval, R):
    global Iter
    try:
        m.neval.append(m.neval[-1]+1)
        m.niter.append(Iter)
        m.Ltrain.append(Ltrain)
        m.R.append(R)
        m.Lval.append(Lval)
        
    except:
        # for the first loss evaluation. First evaluation corresponds loss after 0 iterations. After that. Iter is set to 1.
        m.Ltrain.append(Ltrain)
        m.R.append(R)
        m.Lval.append(Lval)
        
        Iter=0
        m.niter.append(Iter)
        Iter=1
        m.neval.append(1) 
    
    plt.figure()
    xx = pd.DataFrame()
    xx['niter']=np.array(m.niter)
    xx['norm']=np.array(m.R)
    xx['Lval']=np.array(m.Lval)
    xx['Ltrain'] = m.Ltrain
    xx = xx.groupby('niter', as_index=False).min() #if multiple function evaluations at same iteration
    plt.plot(xx.niter, xx.Ltrain/xx.Ltrain[0], label='L',c='k')
    plt.plot(xx.niter, xx.Lval/xx.Lval[0], label='Lval',c='k',linestyle='dashed')
    try:
        plt.plot(xx.niter, xx.norm/xx.norm[len(xx.norm)-1], label='norm')
    except:
        print(xx.norm)
    plt.legend()
    plt.yscale("log")
    plt.xscale("log")
    plt.savefig('optimization_synthetic')
    plt.close()
    
    #print(Iter, Ltrain, R, Lval)

In [7]:
# x is logit, z is label
def sigmoidcrossentropy(x,z):#,β):
    #x = β*x
    ma = np.zeros(x.shape)
    ma[x>0]=x[x>0]
    return ma - x*z + np.log(1+np.exp(-abs(x)))
def sigmoidcrossentropy_deriv(x,z):#,β):
    #x = β*x
    res = np.zeros(x.shape)
    res[x>=0] = (1 - z - 1./(1+np.exp(-x))*np.exp(-x))[x>=0]
    res[x<0] = (-z + 1/(1+np.exp(x))*np.exp(x))[x<0]
    return res
def Epos(f, θ=1):
    res =  np.log(1+np.exp(θ*f))/θ
    res[(θ*f)>100]=f[(θ*f)>100]   #if overflow
    return res
def Epos_deriv(f, θ=1):
    res = np.exp(θ*f)/(1+np.exp(θ*f))
    res[(θ*f)>100]=1  #if overflow
    return res

def expcrossentropy(f,y):#,β):
    #f = β*f
    f=f+1e-15
    assert f.all()>0
    return y*f -(1-y)*np.log(1-np.exp(-f))

def expcrossentropy_deriv(f,y):#,β):
    #f = β*f
    f=f+1e-15
    assert f.all()>0
    return y - (1-y)/(1-np.exp(-f))*np.exp(-f)

In [8]:
evaluationdef predict(A, th1, th2, α,β, K1, K2, directsum=False):
    if directsum:
        f = β*(np.matmul(np.matmul(K1, A),K2.transpose()) + outersum(th1, th2))
    else:
        f = β*(np.matmul(np.matmul(K1, A),K2.transpose()) + maxoutersum(th1, th2, α))
    return f

def loss_deriv_(A_,K1,K2,Ytrain,Yval, m):
    if m.version == sigmoidcrossentropy:
        A = A_[:m.A.size].reshape(m.A.shape)
        th1 = A_[m.A.size:m.A.size+m.th1.size].reshape(m.th1.shape)
        th2 = A_[m.A.size+m.th1.size:m.A.size+m.th1.size+m.th2.size].reshape(m.th1.shape)
        β =  A_[m.A.size+m.th1.size+m.th2.size:]
        #f = predict(A, th1, th2, m.α, K1, K2)
        E = np.zeros(Ytrain.shape)
        for stage in range(E.shape[2]):
            E[:,:,stage] =predict(A[:,:,stage], th1[:,stage], th2[:,stage],m.α,β[stage],K1, K2, m.directsum)
        Ep = Epos(E)
        F = np.cumsum(Ep, axis=2)

        DLDF = (np.nan_to_num(sigmoidcrossentropy_deriv(F,Ytrain), nan=0)/len(np.where(~np.isnan(Ytrain))[0])) #if no label is nan, contribution to deriv is zero
        #DLDEp = np.cumsum(np.flip(DLDF,axis=2),axis=2)
        DLDEp = np.flip(np.cumsum(np.flip(DLDF,axis=2),axis=2),axis=2)
        DLDE = DLDEp*Epos_deriv(E)
                
        DLDA = np.zeros(A.shape)
        DLDTH1 = np.zeros(th1.shape)
        DLDTH2 = np.zeros(th1.shape)
        DLDβ = np.zeros(β.shape)
        for stage in range(E.shape[2]):
            DLDA[:,:,stage] = β[stage]*np.matmul(np.matmul(K1.transpose(), DLDE[:,:,stage]), K2)
            if m.directsum:
                DLDTH1[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*outersum_deriv(th1[:,stage],th2[:,stage],m.α)[0], axis=1)
                DLDTH2[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*outersum_deriv(th1[:,stage],th2[:,stage],m.α)[1], axis=0)
            else:
                DLDTH1[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*maxoutersum_deriv(th1[:,stage],th2[:,stage],m.α)[0], axis=1)
                DLDTH2[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*maxoutersum_deriv(th1[:,stage],th2[:,stage],m.α)[1], axis=0)
            if m.varβ:
                DLDβ[stage] = np.sum(DLDE[:,:,stage]*E[:,:,stage]/β[stage])
        DRDA = m.λ*2*A/A.size

        if m.competition:
            return np.concatenate([(DLDA+DRDA).flatten(),DLDTH1.flatten(), DLDTH2.flatten(),DLDβ])
        else:
            return np.concatenate([(DLDA+DRDA).flatten(),np.zeros(DLDTH1.size), np.zeros(DLDTH2.size),DLDβ])
        #DLDF = np.nan_to_num(sigmoidcrossentropy_deriv(f,Ytrain,m.β), nan=0)/len(np.where(~np.isnan(Ytrain))[0]) #if no label is nan, contribution to deriv is zero
    
    elif m.version ==  expcrossentropy:
        A = A_[:m.A.size].reshape(m.A.shape)
        th1 = A_[m.A.size:m.A.size+m.th1.size].reshape(m.th1.shape)
        th2 = A_[m.A.size+m.th1.size:m.A.size+m.th1.size+m.th2.size].reshape(m.th1.shape)
        β =  A_[m.A.size+m.th1.size+m.th2.size:]
        #f = predict(A, th1, th2, m.α, K1, K2)
        E = np.zeros(Ytrain.shape)
        for stage in range(E.shape[2]):
            E[:,:,stage] =predict(A[:,:,stage], th1[:,stage], th2[:,stage],m.α,β[stage],K1, K2, m.directsum)
        Ep = Epos(E)
        F = np.cumsum(Ep, axis=2)

        DLDF = (np.nan_to_num(expcrossentropy_deriv(F,Ytrain), nan=0)/len(np.where(~np.isnan(Ytrain))[0])) #if no label is nan, contribution to deriv is zero
        #DLDEp = np.cumsum(np.flip(DLDF,axis=2),axis=2)
        DLDEp = np.flip(np.cumsum(np.flip(DLDF,axis=2),axis=2),axis=2)
        DLDE = DLDEp*Epos_deriv(E)
                
        DLDA = np.zeros(A.shape)
        DLDTH1 = np.zeros(th1.shape)
        DLDTH2 = np.zeros(th1.shape)
        DLDβ = np.zeros(β.shape)
        for stage in range(E.shape[2]):
            DLDA[:,:,stage] = β[stage]*np.matmul(np.matmul(K1.transpose(), DLDE[:,:,stage]), K2)
            if m.directsum:
                DLDTH1[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*outersum_deriv(th1[:,stage],th2[:,stage],m.α)[0], axis=1)
                DLDTH2[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*outersum_deriv(th1[:,stage],th2[:,stage],m.α)[1], axis=0)
            else:
                DLDTH1[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*maxoutersum_deriv(th1[:,stage],th2[:,stage],m.α)[0], axis=1)
                DLDTH2[:,stage] = β[stage]*np.sum(DLDE[:,:,stage]*maxoutersum_deriv(th1[:,stage],th2[:,stage],m.α)[1], axis=0)
            if m.varβ:
                DLDβ[stage] = np.sum(DLDE[:,:,stage]*E[:,:,stage]/β[stage])
        DRDA = m.λ*2*A/A.size

        if m.competition:
            return np.concatenate([(DLDA+DRDA).flatten(),DLDTH1.flatten(), DLDTH2.flatten(),DLDβ])
        else:
            return np.concatenate([(DLDA+DRDA).flatten(),np.zeros(DLDTH1.size), np.zeros(DLDTH2.size),DLDβ])
    else:
        print('not implemented')
        return 'not implemented'

    
def loss_(A_,K1,K2,Ytrain,Yval, m):
    if m.version == sigmoidcrossentropy:
        A = A_[:m.A.size].reshape(m.A.shape)
        th1 = A_[m.A.size:m.A.size+m.th1.size].reshape(m.th1.shape)
        th2 = A_[m.A.size+m.th1.size:m.A.size+m.th1.size+m.th2.size].reshape(m.th1.shape)
        β =  A_[m.A.size+m.th1.size+m.th2.size:]
        #f = predict(A, th1, th2, m.α, K1, K2)
        E = np.zeros(Ytrain.shape)
        for stage in range(E.shape[2]):
            E[:,:,stage] = predict(A[:,:,stage], th1[:,stage], th2[:,stage],m.α, β[stage],K1, K2,m.directsum)
        Ep = Epos(E)
        F = np.cumsum(Ep, axis=2)
        
        L_train = np.nanmean(sigmoidcrossentropy(F, Ytrain)) #nan if label is not training label        
        L_val = np.nanmean(sigmoidcrossentropy(F, Yval))
        R = m.λ*np.sum(A*A)/A.size

        m.A[:]=A[:] 
        m.th1[:]=th1[:]
        m.th2[:]=th2[:]
        m.β[:]=β
        visualizeprogression(m, L_train, L_val, R)

        return L_train + R
        #L_train = np.nanmean(sigmoidcrossentropy(f, Ytrain,m.β)) #nan if label is not training label
        #L_val = np.nanmean(sigmoidcrossentropy(f, Yval,m.β))

    elif m.version ==  expcrossentropy:
        A = A_[:m.A.size].reshape(m.A.shape)
        th1 = A_[m.A.size:m.A.size+m.th1.size].reshape(m.th1.shape)
        th2 = A_[m.A.size+m.th1.size:m.A.size+m.th1.size+m.th2.size].reshape(m.th1.shape)
        β =  A_[m.A.size+m.th1.size+m.th2.size:]
        #f = predict(A, th1, th2, m.α, K1, K2)
        E = np.zeros(Ytrain.shape)
        for stage in range(E.shape[2]):
            E[:,:,stage] = predict(A[:,:,stage], th1[:,stage], th2[:,stage],m.α, β[stage],K1, K2, m.directsum)
        Ep = Epos(E)
        F = np.cumsum(Ep, axis=2)
        
        L_train = np.nanmean(expcrossentropy(F, Ytrain)) #nan if label is not training label        
        L_val = np.nanmean(expcrossentropy(F, Yval))
        R = m.λ*np.sum(A*A)/A.size

        m.A[:]=A[:] 
        m.th1[:]=th1[:]
        m.th2[:]=th2[:]
        m.β[:]=β
        visualizeprogression(m, L_train, L_val, R)

        return L_train + R
    else:
        print('not implemented')
        return 'not implemented'

def callback(A):
    global Iter
    Iter+=1

Iter=0

class Model_():
    def __init__(self, λ, competition, version, α, varβ, directsum):
        self.λ=λ
        self.A=0
        self.competition=competition
        self.version=version
        self.α=α
        self.varβ=varβ
        self.directsum = directsum
        self.neval, self.niter, self.Ltrain,self.R,self.Lval= [],[],[],[],[]
        global Iter
        Iter = 0

        
    
    def fit(self,K1,K2,Ytrain,Yval=None, maxiter=20):
        if type(self.A) != int:
            global Iter
            Iter = self.niter[-1]
            print("Continuing with current model parameters.")
            #res = minimize(fun=loss,x0=np.concatenate([self.A.flatten(), self.th1.flatten(), self.th2.flatten()]), args=(K1,K2,Ytrain,Yval,self), jac=loss_deriv, method='L-BFGS-B', options ={'maxiter':maxiter,'ftol': 0, 'gtol': 0}, callback=callback)
        else:
            A0=np.zeros((K1.shape[1], K2.shape[1],Ytrain.shape[2]))
            th1 = np.zeros((K1.shape[0],Ytrain.shape[2]))
            th2 = np.zeros((K2.shape[0],Ytrain.shape[2]))
            β = np.ones(Ytrain.shape[2])
            self.A=A0
            self.th1 = th1
            self.th2 = th2
            self.β=β
            print("New model parameters initialized as zeros.")

        res = minimize(fun=loss_,x0=np.concatenate([self.A.flatten(), self.th1.flatten(), self.th2.flatten(),self.β]), args=(K1,K2,Ytrain,Yval,self), jac=loss_deriv_, method='L-BFGS-B', options ={'maxiter':maxiter,'ftol': 0, 'gtol': 0}, callback=callback)
        A_ = res.x
        self.A = A_[:self.A.size].reshape(self.A.shape)
        self.th1 = A_[self.A.size:self.A.size+self.th1.size].reshape(self.th1.shape)
        self.th2 = A_[self.A.size+self.th1.size:self.A.size+self.th1.size+self.th2.size].reshape(self.th1.shape)
        self.β = A_[self.A.size+self.th1.size+self.th2.size:]
        return res
        
    def predict_proba(self, K1,K2,th=True):
        if self.version == sigmoidcrossentropy:
            E = np.zeros((K1.shape[0],K2.shape[0],self.A.shape[2]))
            for stage in range(E.shape[2]):
                E[:,:,stage] = predict(self.A[:,:,stage], self.th1[:,stage], self.th2[:,stage],self.α,self.β[stage], K1, K2)
            Ep = Epos(E)
            F = np.cumsum(Ep, axis=2)
            return 1 / (1 + np.exp(-F))
        elif self.version ==  expcrossentropy:
            E = np.zeros((K1.shape[0],K2.shape[0],self.A.shape[2]))
            for stage in range(E.shape[2]):
                E[:,:,stage] = predict(self.A[:,:,stage], self.th1[:,stage], self.th2[:,stage],self.α,self.β[stage], K1, K2)
            Ep = Epos(E)
            F = np.cumsum(Ep, axis=2)
            return np.exp(-F)
        else:
            print('not implemented')
            return 'not implemented'

    


In [9]:
def clean(Y):
    #forward cleaning: if nan and following is 1: reset following to nan
    for i in range(1,4):
        Y[:,:,i][(np.isnan(Y[:,:,i-1]))&(Y[:,:,i]==1)]=np.nan
    #backward cleaning: if nan and previous was zero: reset previous to zero
    for i in range(3)[::-1]:
        Y[:,:,i][(np.isnan(Y[:,:,i+1]))&(Y[:,:,i]==0)]=np.nan

In [10]:
th = [125, 25, 5, 1]
#p = [1]*4

In [11]:

Y_data = np.load('../DataPreprocessing/'+"Y_nocompetition"+ '_th{}mu{}cov{}repetition{}.npy'.format(th,0,0,0))


In [13]:
sta = 3
np.sum(Y_data[:,:,sta]==1) / np.sum(Y_data[:,:,sta]==0)

0.0009068215803517988

In [14]:
Y_data[:,:,sta]==1

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

## 1. Model evaluation 

### Expotential cross entropy, no competition

In [ ]:
repetitions = 5
result = np.zeros((repetitions, 12, 4))

for repetition in range(repetitions):
    row=0
    for (μ, cov) in [(0,0), (0,0.5), (1,0)]:
        print(μ, cov) 
        for dataset_name in ["Y_nocompetition"]:
            print(dataset_name)
            Y_data = np.load('../DataPreprocessing/'+dataset_name+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xj = np.load('../DataPreprocessing/'+'Xj'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xv = np.load('../DataPreprocessing/'+'Xv'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))

            #shuffle columns of Xj and Xv
            shufflecolumns = 1
            for i in range(shufflecolumns):
                np.random.shuffle(Xj[:,i])
                np.random.shuffle(Xv[:,i])
            #append a bias feature
            Xj = np.append(Xj, np.array(len(Xj)*[1]).reshape(-1,1), axis=1)
            Xv = np.append(Xv, np.array(len(Xv)*[1]).reshape(-1,1), axis=1)


            # specific train-test split: Ytest1=T, Ytest2=A
            ###################################################################################"
            x,y = np.where(~np.isnan(Y_data[:,:,0]))
            r = np.array(range(len(x)))
            np.random.shuffle(r)
            train, test1, test2 = r[:int(0.4*len(r))], r[int(0.4*len(r)): int(0.7*len(r))], r[int(0.7*len(r)):]
            z = np.random.choice(range(4),len(test1))
            #z = np.random.choice([2],len(test1))


            Y_train = copy.deepcopy(Y_data).astype(float)
            Y_train[x[test1], y[test1],z] = np.nan
            clean(Y_train)
            Y_test1 = copy.deepcopy(Y_data).astype(float)
            Y_test1[~np.isnan(Y_train)]=np.nan
            # remark: if we have a row of 0 0 0 0 or 1 1 1 1 and the last or first stage, respectively, is train data
            # the cleaning assigns the whole row to the test data set (because else trivially imputed). One could argue
            # that then this setting is again more difficult for the very frequently 0 0 0 0, because it is more the matrix 
            # completion setting below; and thus that we should neglect if for this training set. However, doing this, we 
            # would drop the very easy predicted 0 0 0 0 lines and in fact having a distorted image that makes this setting
            # in the end more difficult. Therefore, we do not drop the 0 0 0 0 lines of test set where none is observed in 
            # training.


            ##################################################
            Y_train[x[test2], y[test2]] = np.nan
            Y_test2 = copy.deepcopy(Y_data).astype(float)
            Y_test2[x[train],y[train]]=np.nan
            Y_test2[x[test1],y[test1]]=np.nan
            ############################################################################################


            a=time.time()
            model1_=Model_(λ=0, competition=False, version=expcrossentropy,α=1, varβ=False, directsum=False)
            res = model1_.fit(Xj,Xv,Y_train,Y_test1,maxiter=50)
            b=time.time()
            
            Pred = model1_.predict_proba(Xj,Xv)

            result[repetition,row,:] = evaluate_direct(Y_test1, Pred)
            row+=1
            print(row)
            result[repetition,row,:] = evaluate_direct(Y_test2, Pred)
            row+=1
            print(row)
            print()

            result[repetition, row,:] = evaluate_conditional(Y_test1, Pred)
            row+=1
            print(row)
            result[repetition,row,:] = evaluate_conditional(Y_test2, Pred)
            row+=1
            print(row)


            print()
            print('############')
        
        

print("#################") 
print(result)
print("#######################################")
mean, std = np.mean(result, axis=0), np.std(result, axis=0)

for i in range(len(mean)):
    print("& {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2})  \\\\".format(mean[i,0], std[i,0], mean[i,1], std[i,1],mean[i,2],std[i,2],mean[i,3],std[i,3]))


0 0
Y_nocompetition
New model parameters initialized as zeros.


/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.1679400207782919 0.0 0.22578398897019025
1 0.10795372083828764 0.0 0.13754337328022048
2 0.08069342879117009 0.0 0.09497951245383027
3 0.06257569126382595 0.0 0.084950221772895
4 0.05108296571626026 0.0 0.060386953745408166
5 0.04524921517527059 0.0 0.05419864741446704
6 0.04184774713452799 0.0 0.047594518704413974
7 0.04063434119782893 0.0 0.04812478871230712
8 0.04019862552759799 0.0 0.04670788918459776
9 0.04008581796938127 0.0 0.04607067431604454
10 0.039997389611251044 0.0 0.045590804025668855
11 0.039868995498500905 0.0 0.04518114552363051
12 0.03971742869258732 0.0 0.04486686061139885
13 0.03955115364994539 0.0 0.044884585724925714
14 0.039437419674042434 0.0 0.0456025972307403
15 0.03933433601205041 0.0 0.04551369975573716
16 0.03927272666058331 0.0 0.04539067976484737
17 0.03918323785530561 0.0 0.04490094205949746
18 0.03925279304065545 0.0 0.04669304949184524
18 0.03916301475632867 0.0 0.04542540414579211
19 0.039146362934986044 0.0 0.04523119530290674
20 0.03913094874205

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.16833508067208264 0.0 0.2246967762375735
1 0.1077410256723168 0.0 0.13652224617541697
2 0.08056718247881502 0.0 0.09308746972929911
3 0.06254459008854374 0.0 0.08330576567015573
4 0.050988311518135036 0.0 0.058747138064225525
5 0.04525649514958799 0.0 0.052319417807864005
6 0.04190121973432543 0.0 0.04651169599351032
7 0.040726322771354136 0.0 0.046547046369486395
8 0.04019930299270123 0.0 0.045207842998672365
9 0.04005669743361091 0.0 0.044608586569555214
10 0.039914254472710985 0.0 0.04410208554144448
11 0.039655419323599096 0.0 0.043728135195749356
12 0.03930772965769243 0.0 0.04290014578224994
13 0.03897154925658832 0.0 0.043316834617526286
14 0.03865626382596684 0.0 0.043849265964761706
15 0.038521328953538644 0.0 0.04470253180150209
16 0.038430026193626364 0.0 0.04416937946032173
17 0.03833508279930065 0.0 0.044245475096098844
18 0.03829543953642195 0.0 0.04315627217485699
19 0.038254827752653514 0.0 0.04360766684577902
20 0.038216804001162744 0.0 0.044026761585098086
21 0.03

...

1 0.1681589577906473 0.0 0.2249547757250357
1 0.1083026920455096 0.0 0.13731030069386946
2 0.08159667645530815 0.0 0.09477786646921134
3 0.06394886008581377 0.0 0.08598847307973737
4 0.05294279052558736 0.0 0.06256782580853029
5 0.04760898754902299 0.0 0.055705447939317365
6 0.04467231591009839 0.0 0.049707468757407475
7 0.04353899953778956 0.0 0.0502598146325042
8 0.04321066057869272 0.0 0.049072284370542556
9 0.0431034396962658 0.0 0.04853713162579159
10 0.04298685812075562 0.0 0.04807783904811836
11 0.042777626234780264 0.0 0.04768832193677372
12 0.042514783625915875 0.0 0.047323132957032714
13 0.04223378734796004 0.0 0.04749728264041808
14 0.04209380607127163 0.0 0.04837725002478347
15 0.04196050535683237 0.0 0.04816704109592613
16 0.041894525766515554 0.0 0.04811526018709941
17 0.04179307711787132 0.0 0.047478149443351644
18 0.0419115365422995 0.0 0.050356913177374285
18 0.04176861485818432 0.0 0.04824503813073496
19 0.04174739181043347 0.0 0.048055444101477794
20 0.04172877593496

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.16811172221492968 0.0 0.22493390519162323
1 0.1075547872047047 0.0 0.13670255980256182
2 0.08051440237139673 0.0 0.09328530166185436
3 0.0627286757128333 0.0 0.08358754701615684
4 0.05143610755321972 0.0 0.059522362191500346
5 0.04597240671665919 0.0 0.05340309867285949
6 0.04286595657345899 0.0 0.048118438244618054
7 0.04177549608842245 0.0 0.04781960143053997
8 0.041271079096012714 0.0 0.04661067004925931
9 0.041109443850947794 0.0 0.04595833024633581
10 0.040911415790577575 0.0 0.04533941076561758
11 0.040546566790089125 0.0 0.044716836267962516
12 0.04015955698399817 0.0 0.04387001383770862
13 0.039744532446204926 0.0 0.04431521767718224
14 0.039436384063840896 0.0 0.04484299726957277
15 0.039277089280222374 0.0 0.04578150668025585
16 0.03915486913547439 0.0 0.04476029294575601
17 0.0390523575332794 0.0 0.04526823168300505
18 0.0389638171567431 0.0 0.044847985027700044
19 0.038904305319333136 0.0 0.04493706952069968
20 0.03884690251072312 0.0 0.044682131652298875
21 0.038811671

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.16802121288316146 0.0 0.22536551784803585
1 0.10629857300604299 0.0 0.13586701424403333
2 0.07830468104876323 0.0 0.08883321582578606
3 0.0596911964903966 0.0 0.07684762092008628
4 0.049331398007846895 0.0 0.05616199203036897
5 0.04408728946966545 0.0 0.051315879709331595
6 0.04119455447826374 0.0 0.04686644421089072
7 0.04000984153978007 0.0 0.04605036217681275
8 0.03955006780480732 0.0 0.04557461244656605
9 0.03934186906992169 0.0 0.044840233926369225
10 0.03904666421864888 0.0 0.044327474957363576
11 0.038725513722688495 0.0 0.0427999827193002
12 0.03838240263675799 0.0 0.04319791119947406
13 0.038076581770759774 0.0 0.0437417490179423
14 0.03789764818426106 0.0 0.044535773203417235
15 0.03773895971493776 0.0 0.044397226301573445
16 0.037604443539190055 0.0 0.04401087730561002
17 0.03746410625994602 0.0 0.043803600649609094
18 0.037412045320477504 0.0 0.04337999863596809
19 0.03732949557696072 0.0 0.043706290749070446
20 0.037295004065165886 0.0 0.04374988392215311
21 0.03725734

In [18]:
for i in range(len(mean)):
    print("& {:.2} & {:.2} & {:.2} & {:.2}   \\\\".format(mean[i,0], mean[i,1], mean[i,2],mean[i,3]))


& 0.77 & 0.82 & 0.88 & 0.89   \\
& 0.78 & 0.82 & 0.88 & 0.89   \\
& nan & 0.75 & 0.71 & 0.45   \\
& nan & 0.74 & 0.68 & 0.36   \\
& 0.78 & 0.85 & 0.92 & 0.91   \\
& 0.78 & 0.86 & 0.92 & 0.92   \\
& nan & 0.74 & 0.67 & 0.34   \\
& nan & 0.74 & 0.68 & 0.31   \\
& 0.77 & 0.87 & 0.93 & 0.95   \\
& 0.78 & 0.87 & 0.93 & 0.95   \\
& nan & 0.72 & 0.58 & 0.28   \\
& nan & 0.72 & 0.6 & 0.28   \\


### Logistic cross entropy, no competition
This was performed with a different loss function, i.e. sigmoid on marginal probability. But did not work very well

In [19]:
"""
repetitions = 5
result = np.zeros((repetitions, 12, 4))

for repetition in range(repetitions):
    row=0
    for (μ, cov) in [(0,0), (0,0.5), (1,0)]:
        print(μ, cov) 
        for dataset_name in ["Y_nocompetition"]:
            print(dataset_name)
            Y_data = np.load('../DataPreprocessing/'+dataset_name+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xj = np.load('../DataPreprocessing/'+'Xj'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xv = np.load('../DataPreprocessing/'+'Xv'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))

            #shuffle columns of Xj and Xv
            shufflecolumns = 1
            for i in range(shufflecolumns):
                np.random.shuffle(Xj[:,i])
                np.random.shuffle(Xv[:,i])
            #append a bias feature
            Xj = np.append(Xj, np.array(len(Xj)*[1]).reshape(-1,1), axis=1)
            Xv = np.append(Xv, np.array(len(Xv)*[1]).reshape(-1,1), axis=1)


            # specific train-test split: Ytest1=T, Ytest2=A
            ###################################################################################"
            x,y = np.where(~np.isnan(Y_data[:,:,0]))
            r = np.array(range(len(x)))
            np.random.shuffle(r)
            train, test1, test2 = r[:int(0.4*len(r))], r[int(0.4*len(r)): int(0.7*len(r))], r[int(0.7*len(r)):]
            z = np.random.choice(range(4),len(test1))
            #z = np.random.choice([2],len(test1))


            Y_train = copy.deepcopy(Y_data).astype(float)
            Y_train[x[test1], y[test1],z] = np.nan
            clean(Y_train)
            Y_test1 = copy.deepcopy(Y_data).astype(float)
            Y_test1[~np.isnan(Y_train)]=np.nan
            # remark: if we have a row of 0 0 0 0 or 1 1 1 1 and the last or first stage, respectively, is train data
            # the cleaning assigns the whole row to the test data set (because else trivially imputed). One could argue
            # that then this setting is again more difficult for the very frequently 0 0 0 0, because it is more the matrix 
            # completion setting below; and thus that we should neglect if for this training set. However, doing this, we 
            # would drop the very easy predicted 0 0 0 0 lines and in fact having a distorted image that makes this setting
            # in the end more difficult. Therefore, we do not drop the 0 0 0 0 lines of test set where none is observed in 
            # training.


            ##################################################
            Y_train[x[test2], y[test2]] = np.nan
            Y_test2 = copy.deepcopy(Y_data).astype(float)
            Y_test2[x[train],y[train]]=np.nan
            Y_test2[x[test1],y[test1]]=np.nan
            ############################################################################################


            a=time.time()
            model1_=Model_(λ=0, competition=False, version=sigmoidcrossentropy,α=1, varβ=False)
            res = model1_.fit(Xj,Xv,Y_train,Y_test1,maxiter=50)
            b=time.time()
            
            Pred = model1_.predict_proba(Xj,Xv)

            result[repetition,row,:] = evaluate_direct(Y_test1, Pred)
            row+=1
            print(row)
            result[repetition,row,:] = evaluate_direct(Y_test2, Pred)
            row+=1
            print(row)
            print()

            result[repetition, row,:] = evaluate_conditional(Y_test1, Pred)
            row+=1
            print(row)
            result[repetition,row,:] = evaluate_conditional(Y_test2, Pred)
            row+=1
            print(row)


            print()
            print('############')
        
        

print("#################") 
print(result)
print("#######################################")
mean, std = np.mean(result, axis=0), np.std(result, axis=0)

for i in range(len(mean)):
    print("& {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2})  \\\\".format(mean[i,0], std[i,0], mean[i,1], std[i,1],mean[i,2],std[i,2],mean[i,3],std[i,3]))

print("###################")

for i in range(len(mean)):
    print("& {:.2} & {:.2} & {:.2} & {:.2}   \\\\".format(mean[i,0], mean[i,1], mean[i,2],mean[i,3]))
"""

'\nrepetitions = 5\nresult = np.zeros((repetitions, 12, 4))\n\nfor repetition in range(repetitions):\n    row=0\n    for (μ, cov) in [(0,0), (0,0.5), (1,0)]:\n        print(μ, cov) \n        for dataset_name in ["Y_nocompetition"]:\n            print(dataset_name)\n            Y_data = np.load(\'../DataPreprocessing/\'+dataset_name+ \'_th{}mu{}cov{}repetition{}.npy\'.format(th,μ,cov,repetition))\n            Xj = np.load(\'../DataPreprocessing/\'+\'Xj\'+ \'_th{}mu{}cov{}repetition{}.npy\'.format(th,μ,cov,repetition))\n            Xv = np.load(\'../DataPreprocessing/\'+\'Xv\'+ \'_th{}mu{}cov{}repetition{}.npy\'.format(th,μ,cov,repetition))\n\n            #shuffle columns of Xj and Xv\n            shufflecolumns = 1\n            for i in range(shufflecolumns):\n                np.random.shuffle(Xj[:,i])\n                np.random.shuffle(Xv[:,i])\n            #append a bias feature\n            Xj = np.append(Xj, np.array(len(Xj)*[1]).reshape(-1,1), axis=1)\n            Xv = np.append(Xv

## 2. Effect of competition and correlation

In [20]:
repetitions = 5
result = np.zeros((repetitions, 18, 4))

for repetition in range(repetitions):
    row=0
    for (μ, cov) in [(0,0), (0,0.5), (1,0)]:
        print(μ, cov) 
        for dataset_name in ["Y_nocompetition", "Y_softmutualcompetitio", "Y_hardmutualcompetition"]:
            print(dataset_name)
            Y_data = np.load('../DataPreprocessing/'+dataset_name+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xj = np.load('../DataPreprocessing/'+'Xj'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xv = np.load('../DataPreprocessing/'+'Xv'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))

            #shuffle columns of Xj and Xv
            shufflecolumns = 1
            for i in range(shufflecolumns):
                np.random.shuffle(Xj[:,i])
                np.random.shuffle(Xv[:,i])
            #append a bias feature
            Xj = np.append(Xj, np.array(len(Xj)*[1]).reshape(-1,1), axis=1)
            Xv = np.append(Xv, np.array(len(Xv)*[1]).reshape(-1,1), axis=1)


            # specific train-test split: Ytest :setting A
            ###################################################################################
            test_size=0.3
            x,y = np.where(~np.isnan(Y_data[:,:,0]))
            train, test = train_test_split(range(len(x)), test_size=test_size)
            Y_train = copy.deepcopy(Y_data).astype(float)
            Y_test = copy.deepcopy(Y_data).astype(float)
            Y_train[x[test], y[test]] = np.nan
            Y_test[x[train], y[train]] = np.nan
            
            a=time.time()
            model1_=Model_(λ=0, competition=False, version=expcrossentropy,α=1, varβ=False, directsum=False)
            res = model1_.fit(Xj,Xv,Y_train,Y_test,maxiter=50)
            b=time.time()
            
            Pred = model1_.predict_proba(Xj,Xv)


            result[repetition,row,:] = evaluate_direct(Y_test, Pred)
            row+=1
            print(row)
            result[repetition,row,:] = evaluate_conditional(Y_test, Pred)
            row+=1
            print(row)
            print()


            print()
            print('############')
        
        

print("#################") 
print(result)
print("#######################################")
mean, std = np.mean(result, axis=0), np.std(result, axis=0)

for i in range(len(mean)):
    print("& {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2})  \\\\".format(mean[i,0], std[i,0], mean[i,1], std[i,1],mean[i,2],std[i,2],mean[i,3],std[i,3]))


print("###################")

for i in range(len(mean)):
    print("& {:.2} & {:.2} & {:.2} & {:.2}   \\\\".format(mean[i,0], mean[i,1], mean[i,2],mean[i,3]))

0 0
Y_nocompetition
New model parameters initialized as zeros.


/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.18280799728984376 0.0 0.18287988208067077
1 0.11392495593053764 0.0 0.11394540696597551
2 0.08589783593844147 0.0 0.08584479286595459
3 0.0662635907366318 0.0 0.06637394696137741
4 0.05364882676060831 0.0 0.053699682601106004
5 0.04704562583571317 0.0 0.047205489958943074
6 0.04331742148233776 0.0 0.04357486715570059
7 0.0416641702535244 0.0 0.04203081014046291
8 0.04094059954879844 0.0 0.041371537431273024
9 0.040753991945581505 0.0 0.041209508068492384
10 0.04068126544696495 0.0 0.04114259642262606
11 0.04059494591171834 0.0 0.041057855580485976
12 0.04048269939071934 0.0 0.040947936111206465
13 0.04032256952174602 0.0 0.04079955824560307
14 0.040170484590133525 0.0 0.040680017977134234
15 0.04003056868203656 0.0 0.040576088434533246
16 0.0399636671321767 0.0 0.04052100632008685
17 0.03989928476547963 0.0 0.04046592046224697
18 0.039819255923940026 0.0 0.040410319110494634
19 0.03980547261559009 0.0 0.0404407844376656
20 0.039752736526131247 0.0 0.040378038543809
21 0.03974383030

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.18271759087121167 0.0 0.18309083039081242
1 0.12262715111364367 0.0 0.12308543599911093
2 0.1054728063506615 0.0 0.10591446502080598
3 0.09604747865263219 0.0 0.0965443288598158
4 0.09266019664500594 0.0 0.0931561026486617
5 0.09127282387207096 0.0 0.09173433736725191
6 0.0909268753585537 0.0 0.0914383431361884
7 0.09065941874829384 0.0 0.09120681640068978
8 0.09029557959629372 0.0 0.09088037268481654
9 0.08987709480774003 0.0 0.09050717218076679
10 0.08957924550865885 0.0 0.09023825458710771
11 0.08948701973727231 0.0 0.09014330897784598
12 0.08945741817503396 0.0 0.09013210643724996
13 0.08943033386277846 0.0 0.09011143621140894
14 0.08938684395255865 0.0 0.09008920131977606
15 0.0893646279553116 0.0 0.09006866045228418
16 0.08932982383302056 0.0 0.09003461691619453
17 0.08930542340675168 0.0 0.09001131029029094
18 0.08928265296331145 0.0 0.08998854726269019
19 0.08926504250209222 0.0 0.08998494985715677
20 0.08923399386443845 0.0 0.08994997051380989
21 0.08921228251824412 0.0 0.

...

1 0.18293802102002013 0.0 0.18271238189227942
1 0.1116501733744646 0.0 0.1114835899971093
2 0.08179279309639252 0.0 0.08164090522848326
3 0.06190901637601167 0.0 0.06201418775716218
4 0.049016464429599824 0.0 0.04921920401855355
5 0.04321171113535524 0.0 0.04355065299656228
6 0.03934675642961418 0.0 0.0398504711876532
7 0.03766540852877926 0.0 0.038305274061624484
8 0.03698299298146141 0.0 0.03767579146478741
9 0.03665582126832189 0.0 0.03740115645422423
10 0.03640434708894032 0.0 0.03716644749340274
11 0.03611490669864174 0.0 0.03687022329588579
12 0.03581222723261575 0.0 0.036527684459775965
13 0.03556310090266484 0.0 0.03622126598805786
14 0.03536836337745391 0.0 0.0360212416385674
15 0.03525627689757949 0.0 0.035934741026516524
16 0.03517896247036493 0.0 0.03587848910926821
17 0.03505165828832765 0.0 0.03583321682012198
18 0.03497149611683242 0.0 0.035781142125892215
19 0.03490351509902022 0.0 0.035727283664972194
20 0.034840285595077104 0.0 0.035683248964856765
21 0.03478395481275

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.18279759975338788 0.0 0.1830400315144213
1 0.1268888258016018 0.0 0.12726638962411027
2 0.11412774499311804 0.0 0.11460383971534994
3 0.10791886203732896 0.0 0.10853749375257037
4 0.10600211956998604 0.0 0.10679724024308059
5 0.10520128885744628 0.0 0.10603564072419004
6 0.1049326528101178 0.0 0.10579122160579392
7 0.10445203110589232 0.0 0.10535228529991611
8 0.10400086960463943 0.0 0.10492124915161466
9 0.10357282144770986 0.0 0.10453780725880772
10 0.10350634630691052 0.0 0.10447710982404677
11 0.10346971446200606 0.0 0.10445562325059268
12 0.10341959276390991 0.0 0.10442773878821447
13 0.10336223880981012 0.0 0.10440551365748832
14 0.10331391099431571 0.0 0.10438012088874238
15 0.10326624574551124 0.0 0.10433402999911741
16 0.10323667356935352 0.0 0.10428584858463782
17 0.10320088592382393 0.0 0.10424733751459722
18 0.1031854960248508 0.0 0.10423108269397399
19 0.1031588461508842 0.0 0.10420554982334704
20 0.10313808049089483 0.0 0.10419715090873909
21 0.10310778415649345 0.0 0

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.18290028499254987 0.0 0.1828004326230428
1 0.11910935604910196 0.0 0.11895008054228548
2 0.09743297402068068 0.0 0.09728132083194965
3 0.08420695082924885 0.0 0.08407708114248805
4 0.0779173258833029 0.0 0.07791136936376733
5 0.075508479896485 0.0 0.07557033834000935
6 0.07482054698203949 0.0 0.07489454097557613
7 0.07465738246426938 0.0 0.07476827787140435
8 0.07457237156323726 0.0 0.07469635723872965
9 0.07442135757279031 0.0 0.07457593594915891
10 0.07428817098435615 0.0 0.07447988238540176
11 0.07418362805006569 0.0 0.07442030503973905
12 0.07412154432668328 0.0 0.07437487429090449
13 0.0741026667008244 0.0 0.0743564924652142
14 0.07408416868779728 0.0 0.07433956504700774
15 0.07406296675445577 0.0 0.0743222175210635
16 0.07404773983535907 0.0 0.07433613335336586
17 0.07402639046946839 0.0 0.07431450737346126
18 0.07401435785113648 0.0 0.07431025499182842
19 0.07399887680853705 0.0 0.07430859482107315
20 0.07399790543677963 0.0 0.07433406915710365
20 0.07398792655380783 0.0 0.0

In [21]:
mean, std = np.mean(result, axis=0), np.std(result, axis=0)
mean

array([[0.778, 0.826, 0.887, 0.929],
       [  nan, 0.745, 0.705, 0.501],
       [0.275, 0.297, 0.396, 0.477],
       [  nan, 0.248, 0.254, 0.136],
       [0.559, 0.503, 0.517, 0.44 ],
       [  nan, 0.359, 0.241, 0.066],
       [0.78 , 0.855, 0.92 , 0.936],
       [  nan, 0.743, 0.668, 0.441],
       [0.266, 0.342, 0.468, 0.555],
       [  nan, 0.232, 0.187, 0.108],
       [0.563, 0.527, 0.558, 0.508],
       [  nan, 0.323, 0.174, 0.053],
       [0.777, 0.869, 0.934, 0.96 ],
       [  nan, 0.729, 0.608, 0.308],
       [0.153, 0.208, 0.296, 0.415],
       [  nan, 0.232, 0.216, 0.156],
       [0.441, 0.398, 0.441, 0.393],
       [  nan, 0.269, 0.202, 0.063]])

In [22]:
mean, std = np.mean(result, axis=0), np.std(result, axis=0)

for i in range(len(mean)):
    print("& {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2})  \\\\".format(mean[i,0], std[i,0], mean[i,1], std[i,1],mean[i,2],std[i,2],mean[i,3],std[i,3]))


& 0.78 (0.026) & 0.83 (0.02) & 0.89 (0.02) & 0.93 (0.011)  \\
& nan (nan) & 0.75 (0.031) & 0.71 (0.034) & 0.5 (0.057)  \\
& 0.27 (0.011) & 0.3 (0.012) & 0.4 (0.018) & 0.48 (0.026)  \\
& nan (nan) & 0.25 (0.023) & 0.25 (0.012) & 0.14 (0.019)  \\
& 0.56 (0.016) & 0.5 (0.017) & 0.52 (0.012) & 0.44 (0.015)  \\
& nan (nan) & 0.36 (0.023) & 0.24 (0.013) & 0.066 (0.0056)  \\
& 0.78 (0.016) & 0.85 (0.011) & 0.92 (0.012) & 0.94 (0.0089)  \\
& nan (nan) & 0.74 (0.025) & 0.67 (0.031) & 0.44 (0.042)  \\
& 0.27 (0.017) & 0.34 (0.013) & 0.47 (0.021) & 0.55 (0.017)  \\
& nan (nan) & 0.23 (0.024) & 0.19 (0.018) & 0.11 (0.017)  \\
& 0.56 (0.014) & 0.53 (0.0089) & 0.56 (0.0072) & 0.51 (0.019)  \\
& nan (nan) & 0.32 (0.024) & 0.17 (0.0066) & 0.053 (0.0096)  \\
& 0.78 (0.027) & 0.87 (0.016) & 0.93 (0.0083) & 0.96 (0.0061)  \\
& nan (nan) & 0.73 (0.053) & 0.61 (0.036) & 0.31 (0.043)  \\
& 0.15 (0.008) & 0.21 (0.01) & 0.3 (0.0088) & 0.41 (0.025)  \\
& nan (nan) & 0.23 (0.014) & 0.22 (0.013) & 0.16 (0.021)  

## 3. Improving with competition model

In [ ]:
repetitions = 5
result = np.zeros((repetitions, 12, 4))

for repetition in range(repetitions):
    row=0
    for (μ, cov) in [(0,0), (0,0.5), (1,0)]:
        print(μ, cov) 
        for dataset_name in ["Y_softmutualcompetitio"]:
            print(dataset_name)
            Y_data = np.load('../DataPreprocessing/'+dataset_name+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xj = np.load('../DataPreprocessing/'+'Xj'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))
            Xv = np.load('../DataPreprocessing/'+'Xv'+ '_th{}mu{}cov{}repetition{}.npy'.format(th,μ,cov,repetition))

            #shuffle columns of Xj and Xv
            shufflecolumns = 1
            for i in range(shufflecolumns):
                np.random.shuffle(Xj[:,i])
                np.random.shuffle(Xv[:,i])
            #append a bias feature
            Xj = np.append(Xj, np.array(len(Xj)*[1]).reshape(-1,1), axis=1)
            Xv = np.append(Xv, np.array(len(Xv)*[1]).reshape(-1,1), axis=1)


            # specific train-test split: Ytest :setting A
            ###################################################################################
            test_size=0.3
            x,y = np.where(~np.isnan(Y_data[:,:,0]))
            train, test = train_test_split(range(len(x)), test_size=test_size)
            Y_train = copy.deepcopy(Y_data).astype(float)
            Y_test = copy.deepcopy(Y_data).astype(float)
            Y_train[x[test], y[test]] = np.nan
            Y_test[x[train], y[train]] = np.nan
            
            a=time.time()
            model1_=Model_(λ=0, competition=False, version=expcrossentropy,α=10, varβ=False, directsum=False)
            res = model1_.fit(Xj,Xv,Y_train,Y_test,maxiter=50)
            b=time.time()
            
            
            a=time.time()
            model2_=Model_(λ=1e0, competition=True, version=expcrossentropy,α=10, varβ=True, directsum=False)
            res = model2_.fit(Xj,Xv,Y_train,Y_test,maxiter=600)

            b=time.time()
            
            Pred1 = model1_.predict_proba(Xj,Xv)
            Pred2 = model2_.predict_proba(Xj,Xv)


            result[repetition,row,:] = evaluate_direct(Y_test, Pred1)
            row+=1
            print(row)
            result[repetition,row,:] = evaluate_direct(Y_test, Pred2)
            row+=1
            print(row)
            
            result[repetition,row,:] = evaluate_conditional(Y_test, Pred1)
            row+=1
            print(row)
            result[repetition,row,:] = evaluate_conditional(Y_test, Pred2)
            row+=1
            print(row)
            print()


            print()
            print('############')
        
        

print("#################") 
print(result)
print("#######################################")
mean, std = np.mean(result, axis=0), np.std(result, axis=0)

for i in range(len(mean)):
    print("& {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2})  \\\\".format(mean[i,0], std[i,0], mean[i,1], std[i,1],mean[i,2],std[i,2],mean[i,3],std[i,3]))


print("###################")

for i in range(len(mean)):
    print("& {:.2} & {:.2} & {:.2} & {:.2}   \\\\".format(mean[i,0], mean[i,1], mean[i,2],mean[i,3]))

0 0
Y_softmutualcompetitio
New model parameters initialized as zeros.


/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.28762522053757705 0.0 0.28746463245047943
1 0.1666345113290799 0.0 0.16641107534624044
2 0.11878449888930927 0.0 0.11832154031960208
3 0.10361893751049743 0.0 0.10317633810499667
4 0.09541906504751278 0.0 0.09491995708280891
5 0.09255355580210876 0.0 0.09218709195613455
6 0.09161748848536942 0.0 0.0912319059802057
7 0.09120875264884998 0.0 0.09081384062855924
8 0.09090557529504446 0.0 0.09051106902199957
9 0.09054353464852997 0.0 0.0901985757856157
10 0.09011925875695803 0.0 0.08980677984226682
11 0.0899030611378732 0.0 0.08962968714355313
12 0.08984019911442548 0.0 0.08959735976989057
13 0.08978772193600626 0.0 0.08955714389988838
14 0.08975556418024236 0.0 0.0895304636699984
15 0.08970542057023968 0.0 0.08948907573483593
16 0.0896818370675531 0.0 0.08948418454989637
17 0.0896544880972138 0.0 0.0894514505142409
18 0.08962595538983877 0.0 0.08942099675269582
19 0.08960008917722688 0.0 0.0893983600720644
20 0.08955408966079015 0.0 0.08937255186303932
21 0.0895346029273166 0.0 0.0893

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.28762522053757705 0.0 0.28746463245047943
1 0.16240276408805823 0.0005642432689840854 0.16216926559932066
2 0.12723605340112232 0.0033924401316604807 0.1261083484195575
3 0.09813235193764808 0.002972455697388412 0.09759725104233778
4 0.09638474436734171 0.002873526750022574 0.09588882518269345
5 0.09455025491310387 0.0028701694408426494 0.09407703909202107
6 0.09334624245927803 0.003019221088374501 0.09289717048409539
7 0.09187875773720004 0.003259933529177447 0.09149299716461458
8 0.09119658912849044 0.0037738607646469265 0.09079853469148606
9 0.09048336576204509 0.0036619017573959745 0.09012207657417322
10 0.09041788320748263 0.003642877354848059 0.09006447160058222
11 0.0902103384748147 0.0035285219631313023 0.08988243875186751
12 0.09006721678527943 0.0033325079588633934 0.08977628614108209
13 0.08980259065700076 0.0030447445499711782 0.08955069643086161
14 0.08966321566554504 0.0027317593175123006 0.08945129386650998
15 0.08911690171151569 0.0026926890549376784 0.0889362904865

...

/opt/Anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: UserWarning: Data has no positive values, and therefore cannot be log-scaled.


1 0.2877412094313151 0.0 0.28728966306087106
1 0.16470333243456978 0.0005641798085473385 0.16413225012072133
2 0.14081457944359568 0.0031540066315972003 0.139214264143633
3 0.11079818655332606 0.002420532395089864 0.1098822593366654
4 0.10914854779222873 0.0022843008446311033 0.10830368794566442
5 0.10786345257756896 0.0022999386748606175 0.10702509004554964
6 0.10620375719797004 0.0025088865241745585 0.10537656170907593
7 0.1050303696577458 0.0026773353911823373 0.10423802426075904
8 0.10384070466235944 0.0030391756419331766 0.1031302903547447
9 0.10339871402136533 0.0031352820270016974 0.10268743806361573
10 0.10320758140024068 0.0031608594031640024 0.10251734183647432
11 0.1028982991327932 0.003162806389926486 0.1022267111548392
12 0.10271882854614024 0.0030714369990583654 0.10211023376212183
13 0.1024815026358064 0.002946431021387618 0.10187949607681056
14 0.10221875754367739 0.0024537780857695223 0.10160791757790877
15 0.10200673567390106 0.002447550977800187 0.10132173681553149
1

121 0.046380373604927715 0.0003395978237583386 0.04749438864113448
122 0.04633648227609303 0.0003355719759269086 0.04743922836037434
123 0.04628180549680173 0.00033817842851975886 0.04738652728392933
124 0.046245960362438845 0.0003395564253737 0.04735206929349856
125 0.046186020385732836 0.00033955362286681366 0.04729290626788533
126 0.046102992671994955 0.0003382016539619061 0.04721384208825733
127 0.04602545059709695 0.0003369435733229565 0.047139826333231966
128 0.04595234723100775 0.0003355674018962584 0.047071250241887096
129 0.04586886060999277 0.0003345647104787075 0.047000747845794125
130 0.04579746086510047 0.0003357690747734604 0.04693593604550573
131 0.04566492021955905 0.0003506185590809072 0.04684314479825771
132 0.04549955450786241 0.00035749762871715665 0.046674965264020395
133 0.045421021174739756 0.000364527061521549 0.04659734999996318
134 0.045358556495188676 0.00037146782475799215 0.046537888306004374
135 0.04523100853195891 0.00038150048299732226 0.0464194119500633

242 0.04000012894250527 0.0005108910404871251 0.041568853125381375
243 0.03998800304308566 0.0005104280255655815 0.041560258760462175
244 0.039982883404446466 0.0005078406586832695 0.041556207178091616
245 0.0399460475911018 0.0005084871353912034 0.04153327609862493
246 0.039931634658500664 0.0005084084646560042 0.04152647274295272
247 0.03991882277415502 0.0005079482814007341 0.04152194463015151
248 0.03990314221000154 0.0005075072156565623 0.04151123076133394
249 0.03987822701619456 0.000507881653066963 0.04149750571775342
250 0.03985069518190698 0.0005083120691828513 0.0414680989029704
251 0.03983795613920153 0.0005083960984453472 0.0414533051610994
252 0.0398214739552611 0.0005087179250470576 0.041434279141633806
253 0.03980661610596081 0.0005089829743028616 0.04141916220325835
254 0.03977425922844423 0.0005094501929529866 0.041398746772998575
255 0.039801729477255646 0.0005092334905612047 0.041402279516913894
255 0.039759877655875936 0.0005091793729265367 0.04137615852299013
256 0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
mean, std = np.mean(result, axis=0), np.std(result, axis=0)
mean

NameError: name 'result' is not defined

In [16]:
mean, std = np.mean(result, axis=0), np.std(result, axis=0)

for i in range(len(mean)):
    print("& {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2}) & {:.2} ({:.2})  \\\\".format(mean[i,0], std[i,0], mean[i,1], std[i,1],mean[i,2],std[i,2],mean[i,3],std[i,3]))


NameError: name 'result' is not defined